In [1]:
import torch

from fedcore.models.backbone.mobilenet import MobileNet
from fedcore.models.backbone.resnet import ResNetModel
from fedcore.models.backbone.resnet import ObjectDetector
from fedcore.architecture.dataset.object_detection_datasets import YOLODataset, KittiCustomDataset, LabelEncoder
from torchvision.transforms import v2

import numpy as np
import torch.nn
import torchvision.datasets
from fedot.core.pipelines.pipeline_builder import PipelineBuilder
from torch import nn, optim
import torch.nn.functional as F 
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18

from fedcore.architecture.comptutaional.devices import default_device
from fedcore.architecture.utils.paths import data_path
from fedcore.data.data import CompressionInputData
from fedcore.inference.onnx import ONNXInferenceModel
from fedcore.neural_compressor.config import Torch2ONNXConfig
from fedcore.repository.constanst_repository import FEDOT_TASK
from fedcore.repository.initializer_industrial_models import FedcoreModels
from fedcore.repository.constanst_repository import CROSS_ENTROPY, MSE

from fedcore.tools.ruler import PerformanceEvaluatorOD

In [2]:
# import the necessary packages
# import torch
import os
import torch


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# define the base path to the input dataset and then use it to derive
# the path to the input images and annotation CSV files
BASE_PATH = "datasets/kitti"
TRAIN_IMAGES_PATH = os.path.altsep.join([BASE_PATH, "train/img/"])
TRAIN_LABELS_PATH = os.path.altsep.join([BASE_PATH, "train/ann/"])
TRAIN_LENGTH = 7480
# define the path to the base output directory
BASE_OUTPUT = "output"
# define the path to the output model, label encoder, plots output
# directory, and testing image paths
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.pth"])
LE_PATH = os.path.sep.join([BASE_OUTPUT, "le.pickle"])
PLOTS_PATH = os.path.sep.join([BASE_OUTPUT, "plots"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

# these are standard deviation and Mean from ImageNet dataset
STD = [0.229, 0.224, 0.225]
MEAN = [0.485, 0.456, 0.406]

# learning rate
INIT_LR = 0.002

# epochs
NUM_EPOCHS = 20

# bbox and label class loss
BBOX = 1.0
LABELS = 1.0

In [ ]:
def collate_fn(batch):
    images = torch.stack([sample[0] for sample in batch], dim=0)
    labels = [F.pad(sample[1]['labels'], (0, 8 - sample[1]['boxes'].shape[0])) for sample in batch]
    labels = torch.stack(labels, dim=0)
    bboxes = [F.pad(sample[1]['boxes'], (0, 0, 0, 8 - sample[1]['boxes'].shape[0])) for sample in batch]
    bboxes = torch.stack(bboxes, dim=0)
    return images, labels, bboxes

In [17]:
if __name__ == '__main__':
    transform = v2.Compose([
        v2.ToPILImage(),
        v2.ToTensor(),
        v2.Normalize(mean=MEAN, std=STD),
        v2.Resize((640, 480))
    ])
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
   
    train_dataset = YOLODataset(path="datasets\coco8\coco8.yaml", transform=transform, train=True)
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=1,
        shuffle=False
    )

    val_dataset = YOLODataset(path="datasets\coco8\coco8.yaml", transform=transform, train=True)
    val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [0.1, 0.9])
    val_dataloader = DataLoader(
        val_dataset, 
        batch_size=1, 
        shuffle=False
    )

    model = resnet18(pretrained=True).to(DEVICE)
    model.fc = nn.Linear(512, 10).to(DEVICE)
    
    # create our custom object detector model and move it to the current device
    objectDetector = ObjectDetector(model, 79)
    objectDetector = objectDetector.to(DEVICE)
    
    bboxLossFunc = MSE()
    classLossFunc = CROSS_ENTROPY()
    opt = Adam(objectDetector.parameters(), lr=INIT_LR)
    model.train()
    # Train the model
    for epoch in tqdm(range(NUM_EPOCHS)):  # loop over the dataset multiple times
        objectDetector.train()
        running_loss = 0.0
        for i, (images, labels, bboxes, _) in enumerate(train_dataloader, 0):
            # send input to device
            (images, labels, bboxes) = (images.to(DEVICE),
			labels.to(default_device()), bboxes.to(DEVICE))
            
            # zero the parameter gradients
            opt.zero_grad()
            
            # forward + backward + optimize
            outputs = objectDetector(images)
            bboxLoss = bboxLossFunc(outputs[0], bboxes)
            
            source_pad = torch.nn.functional.pad(labels, pad=(0, 9 - labels.shape[1], 0, 0))
            source_pad = source_pad.type(torch.float32)
            abs_predictions = torch.abs(outputs[1])
            abs_predictions = abs_predictions.type(torch.float32)
            
            classLoss = classLossFunc(abs_predictions, source_pad)
            totalLoss = (BBOX * bboxLoss) + (LABELS * classLoss)
            
            totalLoss.backward()
            opt.step()

            # print statistics
            running_loss += totalLoss.item()
            if i % 20 == 0:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 20))
                running_loss = 0.0

    evaluator = PerformanceEvaluatorOD(objectDetector, test_dataset, batch_size=1)
    performance = evaluator.eval()
    print('Before quantization')
    print(performance)
    
    model = model.cpu()
    repo = FedcoreModels().setup_repository()
    compression_pipeline = PipelineBuilder().add_node('post_training_quant').build()

    input_data = CompressionInputData(features=np.zeros((2, 2)),
                                      idx=None,
                                      calib_dataloader=val_dataloader,
                                      task=FEDOT_TASK['classification'],
                                      data_type=None,
                                      target=model
                                      )
    
    input_data.supplementary_data.is_auto_preprocessed = True
    compression_pipeline.fit(input_data)
    quant_model = compression_pipeline.predict(input_data).predict

    int8_onnx_config = Torch2ONNXConfig(
        dtype="int8",
        opset_version=16,
        quant_format="QDQ",  # or "QLinear"
        example_inputs=torch.unsqueeze(train_dataset[0][0], dim=0),
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={'input': [0], 'output': [0]}
    )

    quant_model.export("int8-model.onnx", int8_onnx_config)
    onnx_model = ONNXInferenceModel("int8-model.onnx")
    
    evaluator = PerformanceEvaluatorOD(onnx_model,  test_dataset, batch_size=64)
    performance = evaluator.eval()
    print('after quantization')
    print(performance)

  0%|          | 0/20 [00:00<?, ?it/s]

[1,     1] loss: 4860.821


  5%|▌         | 1/20 [00:01<00:22,  1.18s/it]

[2,     1] loss: 4875.154


 10%|█         | 2/20 [00:02<00:21,  1.21s/it]

[3,     1] loss: 4861.874


 15%|█▌        | 3/20 [00:04<00:22,  1.35s/it]


KeyboardInterrupt: 